In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


from normalised_image_dataset import NormalisedImageDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
#pip install soundfile

In [3]:
df = pd.read_csv('data/normalised_image_train_val_with_upsample.csv')
df.head()

,Unnamed: 0,file_path,label
0,0,data/image_train_val_with_upsample/0.pt,56
1,1,data/image_train_val_with_upsample/1.pt,56
2,2,data/image_train_val_with_upsample/2.pt,56
3,3,data/image_train_val_with_upsample/3.pt,56
4,4,data/image_train_val_with_upsample/4.pt,56


In [4]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.02)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [5]:
df_train.shape, df_val.shape

((87548, 3), (1836, 3))

In [6]:
train_dataset = NormalisedImageDataSet(
    df=df_train, 
    shuffle=True
)

val_dataset = NormalisedImageDataSet(
    df=df_val, 
    shuffle=False
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [7]:
device = model_training.get_device()

In [8]:
from torchvision.models import resnet34, ResNet34_Weights
import torch.nn as nn
import torch.optim as optim

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
resnet_model = resnet34()
resnet_model.fc = nn.Linear(512, 66)
resnet_model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet_model = resnet_model.to(device)

In [9]:
optimizer = optim.Adam(resnet_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [10]:
model_training.training(
    model=resnet_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/resnet34_train_and_val", 
    start_epoch=19,
    early_stop_thresh=50,
)

KeyboardInterrupt: 

In [11]:
torch.save(resnet_model, 'models/resnet34_train_and_val/resnet_model_epoch_19.pth')